# Deploy the Fastai Classifier using Gradio and Hugging Face Spaces

# Preparations

In [19]:
# connect colab to google drive by mounting
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
# navigate to WasteWise directory
%cd drive/MyDrive/wt23-wastewise/

[Errno 2] No such file or directory: 'drive/MyDrive/wt23-wastewise/'
/content/drive/MyDrive/wt23-wastewise


In [9]:
# get latest version of branch
!git pull

remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 32 (delta 19), reused 31 (delta 18), pack-reused 0
Unpacking objects: 100% (32/32), 6.39 MiB | 401.00 KiB/s, done.
From https://github.com/TechLabs-Berlin/wt23-wastewise
   8239cd4..d16396d  classifier_fastai      -> origin/classifier_fastai
   6d8b0ff..4c6ee67  wd-frontend-crftwrks-2 -> origin/wd-frontend-crftwrks-2
Already up to date.


In [21]:
# install the fastbook library
!pip install -Uqq fastbook

In [22]:
# install gradio
!pip install -Uqq gradio

In [23]:
# import libraries
# fastbook/fastai related libraries: used for training the classifier
from fastbook import *
from fastai.vision.widgets import *

# import gradio for deployment 
import gradio as gr

In [44]:
# disable warnings
import warnings
warnings.filterwarnings('ignore')

# Load the model

In [80]:
# load the model
learn = load_learner('/content/drive/MyDrive/wastewise_models/waste_recogniser_fastai_v2.pkl', cpu=True)

In [103]:
# test model inference on banana peel and see how long it takes
%time learn.predict("/content/drive/MyDrive/wt23-wastewise/AI/data_20_classes/banana_peels/10_175867801.jpg")

CPU times: user 410 ms, sys: 9.95 ms, total: 420 ms
Wall time: 427 ms


('banana_peels',
 tensor(2),
 tensor([5.1117e-05, 1.8186e-03, 9.7781e-01, 1.1703e-04, 4.7139e-04, 3.4299e-05, 8.4459e-04, 3.7959e-04, 1.2728e-04, 7.4091e-05, 8.2790e-05, 1.5245e-04, 1.0221e-04, 1.7327e-04, 1.0396e-02, 2.2642e-04,
         2.3036e-04, 2.7148e-04, 2.5266e-05, 6.6162e-03]))

The fine tuned network has 101 layers and I was concerned it would take too long for inference, but this is definitely reasonable.

# Trying out Gradio

Make the most basic gradio app in the world.

In [26]:
def greet(name):
    return "Hello " + name + "!"

demo = gr.Interface(fn=greet, inputs="text", outputs="text")

In [27]:
demo.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

This is the most basic gradio app ever, but it demonstrates that it works in Colab, which was the goal right here. Now scale it up!

# Make a gradio app that can be used in an API for the real app.

In this case, I keep the outputs minimal, because the rest will be solved in JavaScript and HTML together with our WebDev.

In [82]:
# make a "list" containing the classes the classifier can distinguish
# note that actual data type is not "list", but "fastai.data.transforms.CategoryMap"
# it still works in a comparable way
labels = learn.dls.vocab

In [41]:
# see labels
print(type(labels))
print(labels)
print("Label at index 2: " + labels[2])

<class 'fastai.data.transforms.CategoryMap'>
['aluminum_foil', 'apples', 'banana_peels', 'cardboard', 'condoms', 'diapers', 'food_waste', 'glass_bottle', 'old_books', 'oranges', 'pans', 'pizza_box', 'plastic_bags', 'plastic_packaging', 'plastic_toys', 'smartphone', 'tampons', 'tea_bags', 'tetrapack', 'toothbrush']
Label at index 2: banana_peels


In [111]:
# make dictionaries translating the class to something else for the output

# adapt class spelling for output
spelling_dict = {
    "aluminum_foil": "aluminum foil",
    "apples": "an apple",
    "banana_peels": "a banana peel",
    "cardboard": "cardboard",
    "condoms": "a condom",
    "diapers": "a diaper",
    "food_waste": "food waste",
    "glass_bottle": "a glass bottle",
    "old_books": "a book",
    "oranges": "an orange",
    "pans": "a pan",
    "pizza_box": "a pizza box",
    "plastic_bags": "a plastic bag",
    "plastic_packaging": "plastic packaging",
    "plastic_toys": "a plastic toy",
    "smartphone": "a smartphone",
    "tampons": "a tampon",
    "tea_bags": "a tea bag",
    "tetrapack": "a tetra pak",
    "toothbrush": "a toothbrush"
    }

    # recommend waste bin for each class
bin_dict = {
    "aluminum_foil": "gelbe sack",
    "apples": "bio waste",
    "banana_peels": "bio waste",
    "cardboard": "paper waste",
    "condoms": "residual waste",
    "diapers": "residual waste",
    "food_waste": "residual waste",
    "glass_bottle": "glass waste",
    "old_books": "paper waste",
    "oranges": "bio waste",
    "pans": "residual waste",
    "pizza_box": "residual waste",
    "plastic_bags": "plastic waste",
    "plastic_packaging": "plastic waste",
    "plastic_toys": "residual waste",
    "smartphone": "wertstoffsammlung",
    "tampons": "residual waste",
    "tea_bags": "bio waste",
    "tetrapack": "plastic waste",
    "toothbrush": "residual waste"}

In [118]:
# define a function for the learner
def predict(img):
    img = PILImage.create(img)
    pred,pred_idx,probs = learn.predict(img)

    return {labels[i]: float(probs[i]) for i in range(len(labels))}

In [119]:
# make a gradio interface
gr.Interface(fn=predict, inputs=gr.inputs.Image(shape=(512, 512)), outputs=gr.outputs.Label(num_top_classes=3)).launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://e78b04af3c4537b587.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


# Make a proper gradio website that can be shown off as a standalone.

In [122]:
# define a function for the learner
def predict(img):
    img = PILImage.create(img)
    pred,pred_idx,probs = learn.predict(img)

    return {labels[i]: float(probs[i]) for i in range(len(labels))}, f'It belongs into the {bin_dict[pred]}.'

In [123]:
# make a gradio interface
gr.Interface(fn=predict, inputs=gr.inputs.Image(shape=(512, 512)), outputs=[gr.outputs.Label(num_top_classes=3), "text"]).launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://a808bc5cb41c100b09.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
